In [1]:
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import pickle

from datetime import datetime, date, time
from textblob import TextBlob
from textblob import Word
from pre_processing import *
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Andres
[nltk_data]     Montero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
stop = stopwords.words('english')
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]


In [3]:
DATA_FOLDER = './data'

tweets_col_names=['text']

tweets_dtypes = {'text': str }

cleaned_pos = False
cleaned_neg = False
cleaned_test = False
cleaned_pos_full = True
cleaned_neg_full = True


In [4]:
#verify  flag for data already cleaned
if not cleaned_neg :
    #read data from .txt to preprocess
    tweets_neg = pd.read_fwf(DATA_FOLDER + '/train_neg.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
    
    #increase index to start at 1
    tweets_neg.index = tweets_neg.index +1
    #remove duplicates
    tweets_neg.drop_duplicates(inplace=True)
    test = tweets_neg.head(50)
    #apply pre_process funtioncs
    #remove stop words
#     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    # interpret emoji
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: interpret_emoji(x))
    #separate number with letters  1234test123 =>  test 
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: split_number_text(x))
    #remove hashtag #
    tweets_neg['text'] = tweets_neg['text'].str.replace('#', '')
    #replace ur by your
    tweets_neg['text'] = tweets_neg['text'].str.replace('ur', 'your')
    #expand contractions don't => do not
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: expand_contractions(x))
    # remove words user, url, number
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_words(x))
    # replace more letters haaaaaaaaaappy => happy
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: replace_moreletters(x))
    
    #remove most frequent words 
    freq_max_neg = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[:10]
    freq_max_neg = list(freq_max_neg.index)
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_neg))
    
    #remove least frequent words
    freq_min_neg = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[-10:]
    freq_min_neg = list(freq_min_neg.index)
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_neg))

    #correct words with textblob
    # tweets_neg['text'] = tweets_neg['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_neg
    
    #lemmatize words
    tweets_neg['text_lema'] = tweets_neg['text'].apply(lemmatize_text)
    tweets_neg['text_lema'] = tweets_neg['text_lema'].apply(lambda x: ' '.join(x))
    tweets_neg['text_lema'] = tweets_neg['text_lema'].str.replace(',','')
    
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_neg_wstp.pickle","wb")
    pickle.dump(tweets_neg['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
    f = open("data/pre_processed/tweets_neg_wstp.txt", "w")
    f.write("\n".join(map(lambda x: str(x), tweets_neg['text_lema'])))
    f.close()
else:
    print("Already Trained!")
    
pickle_in = open("data/pre_processed/tweets_neg_wstp.pickle","rb")
tweets_neg_pickle = pickle.load(pickle_in)
print("Opening pickle")
tweets_neg_pickle.head()
    

Saving file with preprocessed Tweets
Saved!
Opening pickle


1    vinco tresorpack difficulty of object disassem...
2         glad dot have taks tomorrow thankful startho
3    v celtic in regular season were fucked if we p...
4          could actually kill that girl i am so sorry
5                find that very hard believe im afraid
Name: text_lema, dtype: object

In [5]:
if not cleaned_pos:
    #read data .txt file
    tweets_pos = pd.read_fwf(DATA_FOLDER + '/train_pos.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
    # index start at 1
    tweets_pos.index = tweets_pos.index + 1
    #remove duplicates
    tweets_pos.drop_duplicates(inplace=True)
    #apply pre_process funtioncs
    #remove stop words
#     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    # interpret emoji
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: interpret_emoji(x))
    #separate number with letters  1234test123 =>  test 
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: split_number_text(x))
    print(tweets_pos['text'].count())
    #remove hashtag #
    tweets_pos['text'] = tweets_pos['text'].str.replace('#', '')
    #replace ur by your
    tweets_pos['text'] = tweets_pos['text'].str.replace('ur', 'your')
    #expand contractions don't => do not
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: expand_contractions(x))
    # remove words user, url, number
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: remove_words(x))
    # replace more letters haaaaaaaaaappy => happy
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: replace_moreletters(x))
    
    #remove most frequent words 
    freq_max_pos = pd.Series(' '.join(tweets_pos['text']).split()).value_counts()[:10]
    freq_max_pos = list(freq_max_pos.index)
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_pos))
    
    #remove least frequent words
    freq_min_pos = pd.Series(' '.join(tweets_pos['text']).split()).value_counts()[-10:]
    freq_min_pos = list(freq_min_pos.index)
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_pos))

    #correct words with textblob
    # tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_pos
    
    #lemmatize words
    tweets_pos['text_lema'] = tweets_pos['text'].apply(lemmatize_text)
    tweets_pos['text_lema'] = tweets_pos['text_lema'].apply(lambda x: ' '.join(x))
    tweets_pos['text_lema'] = tweets_pos['text_lema'].str.replace(',','')
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_pos_wstp.pickle","wb")
    pickle.dump(tweets_pos['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
    f = open("data/pre_processed/tweets_pos_wstp.txt", "w")
    f.write("\n".join(map(lambda x: str(x), tweets_pos['text_lema'])))
    f.close()
else:
    print("Already Trained!")
pickle_in = open("data/pre_processed/tweets_pos_wstp.pickle","rb")
tweets_pos_pickle = pickle.load(pickle_in)
print("Opening pickle")
tweets_pos_pickle.head()

90233
Saving file with preprocessed Tweets
Saved!
Opening pickle


1    dunno justin read mention or not only justin g...
2    because yoyour logic so dumb will not even cro...
3      just put casper in box looved battle crakkbitch
4    thanks sir do not trip lil mama .. just keep d...
5    visiting brother tmr bestest birthday gift eveerr
Name: text_lema, dtype: object

In [6]:
if not cleaned_test:  
    #read data from .txt
    test_data = pd.read_fwf(DATA_FOLDER + '/test_data.txt', header=None)
    #index start at 1
    test_data.index = test_data.index +1
    #remove duplicates
    test_data.drop_duplicates(inplace=True)
    test_data.drop(columns=[1,2], inplace=True)
    test_data.rename(columns={0:'text'}, inplace= True)
    
    #apply pre_process funtioncs
    #remove stop words
#     test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    # interpret emoji
    test_data['text'] = test_data['text'].apply(lambda x: interpret_emoji(x))
    #separate number with letters  1234test123 => test 
    test_data['text'] = test_data['text'].apply(lambda x: split_number_text(x))
    #remove hashtag #
    test_data['text'] = test_data['text'].str.replace('#', '')
    #replace ur by your
    test_data['text'] = test_data['text'].str.replace('ur', 'your')
    #expand contractions don't => do not
    test_data['text'] = test_data['text'].apply(lambda x: expand_contractions(x))
    # remove words user, url, number
    test_data['text'] = test_data['text'].apply(lambda x: remove_words(x))
    # replace more letters haaaaaaaaaapy => hapy
    test_data['text'] = test_data['text'].apply(lambda x: replace_moreletters(x))
    
    #remove most frequent words 
    freq_max_test = pd.Series(' '.join(test_data['text']).split()).value_counts()[:10]
    freq_max_test = list(freq_max_test.index)
    test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_test))
    
    #remove least frequent words
    freq_min_test = pd.Series(' '.join(test_data['text']).split()).value_counts()[-10:]
    freq_min_test = list(freq_min_test.index)
    test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_test))

    #correct words with textblob
    # tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_pos
    
    #lemmatize words
    test_data['text_lema'] = test_data['text'].apply(lemmatize_text)
    test_data['text_lema'] = test_data['text_lema'].apply(lambda x: ' '.join(x))
    test_data['text_lema'] = test_data['text_lema'].str.replace(',','')
    index = test_data.index.to_series()
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/test_data_wstp.pickle","wb")
    pickle.dump(test_data['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
    f = open("data/pre_processed/test_data_wstp.txt", "w")
    f.write("\n".join(map(lambda x: str(x[0]+1) + ',' + str(x[1]), enumerate(test_data['text_lema']))))
    f.close()
else:
    print("Already Trained!")
pickle_in = open("data/pre_processed/test_data_wstp.pickle","rb")
test_data_pickle = pickle.load(pickle_in)
print("Opening pickle")
test_data_pickle.head()

Saving file with preprocessed Tweets
Saved!
Opening pickle


1    sea doo pro sea scooter sport with portable se...
2    shuck well work all week so now can not come c...
3              i cant stay away from bug thats my baby
4    no madam lol im perfectly fine not contagious ...
5    whenever fall asleep watching tv always wake u...
Name: text_lema, dtype: object

In [7]:
tweets_neg_pickle.head(20)

1     vinco tresorpack difficulty of object disassem...
2          glad dot have taks tomorrow thankful startho
3     v celtic in regular season were fucked if we p...
4           could actually kill that girl i am so sorry
5                 find that very hard believe im afraid
6                   wish could be out all night tonight
7                                    got kicked out wgm
8             rt yes she tell it my lip are closed okay
9                                    why she so perfect
10    hi harry did havea good time in au didnt get s...
11    introduction programming with + + nd edition t...
14                                        i am white aw
15             dan love miss do be sad wheresthegeneral
16      so many wonderful building in dc but still miss
17         it annoying because secretly find it so good
18    post-boom in spanish american fiction suny ser...
19    layer of heart paperback this joyourney wa ins...
21                guess who texted me again want

In [8]:
tweets_pos_pickle.head(20)

1     dunno justin read mention or not only justin g...
2     because yoyour logic so dumb will not even cro...
3       just put casper in box looved battle crakkbitch
4     thanks sir do not trip lil mama .. just keep d...
5     visiting brother tmr bestest birthday gift eveerr
6      yay lifecompleted tweet facebook let know please
7     dnextalbumtitle feel for rollercoaster of life...
8     workin hard or hardly workin rt at hardee's wi...
9                         saw i will be replying in bit
10                                     this were belong
11                                  andd cheer national
12    we send an invitation shop on-line here will f...
13                       just woke up finna go chyourch
14                             agreed more day left tho
15                                  monet with katemelo
16    like damm lexis got lot say when your on twitt...
17    grateful today for dream fulfilled heart so fu...
18                        at home affair shall d

In [9]:
test_data_pickle.head(20)

1     sea doo pro sea scooter sport with portable se...
2     shuck well work all week so now can not come c...
3               i cant stay away from bug thats my baby
4     no madam lol im perfectly fine not contagious ...
5     whenever fall asleep watching tv always wake u...
6     he need get rid of that thing it scare me lol ...
7                         its whatever in terrible mood
8      yess rt thanks jordan love i am gonna call later
9           my friend text me check up on me last night
10    followback please when will your unitytoyour c...
11    watch some of all dumb ass get lock up today h...
12    obsessed with phasell killed it best album eve...
13    robert de niro not gay but with name like lewy...
14    canada have do it in grade but since we do not...
15    please say hi denmark that would be amazing li...
16                                  finally am home now
17    custom pictyoure frame poster frame wide compl...
18      s my new follower mention me for followb

In [10]:
test_data_pickle

1        sea doo pro sea scooter sport with portable se...
2        shuck well work all week so now can not come c...
3                  i cant stay away from bug thats my baby
4        no madam lol im perfectly fine not contagious ...
5        whenever fall asleep watching tv always wake u...
6        he need get rid of that thing it scare me lol ...
7                            its whatever in terrible mood
8         yess rt thanks jordan love i am gonna call later
9              my friend text me check up on me last night
10       followback please when will your unitytoyour c...
11       watch some of all dumb ass get lock up today h...
12       obsessed with phasell killed it best album eve...
13       robert de niro not gay but with name like lewy...
14       canada have do it in grade but since we do not...
15       please say hi denmark that would be amazing li...
16                                     finally am home now
17       custom pictyoure frame poster frame wide compl.

In [11]:
#verify  flag for data already cleaned
if not cleaned_neg_full :
    #read data from .txt to preprocess
    tweets_neg_full = pd.read_fwf(DATA_FOLDER + '/train_neg_full.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
    
    #increase index to start at 1
    tweets_neg_full.index = tweets_neg_full.index +1
    #remove duplicates
    tweets_neg_full.drop_duplicates(inplace=True)
    test = tweets_neg_full.head(50)
    #apply pre_process funtioncs
    #remove stop words
#     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    # interpret emoji
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: interpret_emoji(x))
    #separate number with letters  1234test123 =>  test 
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: split_number_text(x))
    #remove hashtag #
    tweets_neg_full['text'] = tweets_neg_full['text'].str.replace('#', '')
    #replace ur by your
    tweets_neg_full['text'] = tweets_neg_full['text'].str.replace('ur', 'your')
    #expand contractions don't => do not
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: expand_contractions(x))
    # remove words user, url, number
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: remove_words(x))
    # replace more letters haaaaaaaaaappy => happy
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: replace_moreletters(x))
    
    #remove most frequent words 
    freq_max_neg = pd.Series(' '.join(tweets_neg_full['text']).split()).value_counts()[:10]
    freq_max_neg = list(freq_max_neg.index)
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_neg))
    
    #remove least frequent words
    freq_min_neg = pd.Series(' '.join(tweets_neg_full['text']).split()).value_counts()[-10:]
    freq_min_neg = list(tweets_neg_full.index)
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_neg))

    #correct words with textblob
    # tweets_neg['text'] = tweets_neg['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_neg
    
    #lemmatize words
    tweets_neg_full['text_lema'] = tweets_neg_full['text'].apply(lemmatize_text)
    tweets_neg_full['text_lema'] = tweets_neg_full['text_lema'].apply(lambda x: ' '.join(x))
    tweets_neg_full['text_lema'] = tweets_neg_full['text_lema'].str.replace(',','')
    
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_neg_full_wstp.pickle","wb")
    pickle.dump(tweets_neg_full['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
    f = open("data/pre_processed/tweets_neg_full_wstp.txt", "w")
    f.write("\n".join(map(lambda x: str(x), tweets_neg_full['text_lema'])))
    f.close()
else:
    print("Already Trained!")
    
pickle_in = open("data/pre_processed/tweets_neg_full_wstp.pickle","rb")
tweets_neg_full_pickle = pickle.load(pickle_in)
print("Opening pickle")
tweets_neg_full_pickle.head()

Already Trained!


FileNotFoundError: [Errno 2] No such file or directory: 'data/pre_processed/tweets_neg_full_wstp.pickle'

In [ ]:
if not cleaned_pos_full:
    #read data .txt file
    tweets_pos_full = pd.read_fwf(DATA_FOLDER + '/train_pos_full.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
    # index start at 1
    tweets_pos_full.index = tweets_pos_full.index + 1
    #remove duplicates
    tweets_pos_full.drop_duplicates(inplace=True)
    #apply pre_process funtioncs
    #remove stop words
#     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
     # interpret emoji
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: interpret_emoji(x))
    #separate number with letters  1234test123 =>  test 
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: split_number_text(x))
#     print(tweets_pos_full['text'].count())
    #remove hashtag #
    tweets_pos_full['text'] = tweets_pos_full['text'].str.replace('#', '')
    #replace ur by your
    tweets_pos_full['text'] = tweets_pos_full['text'].str.replace('ur', 'your')
    #expand contractions don't => do not
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: expand_contractions(x))
    # remove words user, url, number
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: remove_words(x))
    # replace more letters haaaaaaaaaappy => happy
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: replace_moreletters(x))
    
    #remove most frequent words 
    freq_max_pos = pd.Series(' '.join(tweets_pos_full['text']).split()).value_counts()[:10]
    freq_max_pos = list(freq_max_pos.index)
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_pos))
    
    #remove least frequent words
    freq_min_pos = pd.Series(' '.join(tweets_pos_full['text']).split()).value_counts()[-10:]
    freq_min_pos = list(freq_min_pos.index)
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_pos))

    #correct words with textblob
    # tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_pos
    
    #lemmatize words
    tweets_pos_full['text_lema'] = tweets_pos_full['text'].apply(lemmatize_text)
    tweets_pos_full['text_lema'] = tweets_pos_full['text_lema'].apply(lambda x: ' '.join(x))
    tweets_pos_full['text_lema'] = tweets_pos_full['text_lema'].str.replace(',','')
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_pos_full_wstp.pickle","wb")
    pickle.dump(tweets_pos_full['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
    f = open("data/pre_processed/tweets_pos_full_wstp.txt", "w")
    f.write("\n".join(map(lambda x: str(x), tweets_pos_full['text_lema'])))
    f.close()
else:
    print("Already Trained!")
pickle_in = open("data/pre_processed/tweets_pos_full_wstp.pickle","rb")
tweets_pos_full_pickle = pickle.load(pickle_in)
print("Opening pickle")
tweets_pos_full_pickle.head()